<a href="https://colab.research.google.com/github/krish0309/Claim-Management-in-ML-using-Python-/blob/main/QuantumPythonEncryption1011_Mishra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Spring 2024 Qubit by Qubit's Introduction to Quantum Computing

```



```
Capstone Project Final Submission: Encryption in Python
```

```
By - Krishna Kant Mishra (Student Code- 1011_Mishra)
```

```
Project: Quantum Key Distribution (QKD) with Fernet Encryption using BB84
```


**Description:**
We simulate the Quantum Key Distribution (QKD) process using the BB84 protocol to generate a shared key between Alice and Bob. The shared key is then used for Fernet encryption and decryption, ensuring secure communication.

**Libraries and Technologies Used:**
* Python: Programming language
* Cirq: Quantum computing framework
* cryptography: Fernet encryption library, hashes, and PBKDF2HMAC



**Acknowledgment:**
This project was developed as part of the final capstone project in the “Qubit by Qubit” course taught by The Coding School.

In [ ]:
# Import necessary modules
from cryptography.fernet import Fernet
!pip install cirq --quiet
import cirq
from random import choices
import base64
from cryptography.hazmat.primitives import hashes
!pip install --upgrade cryptography
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [ ]:
#select basis gates and encode gates

basis_gates = {'Z': cirq.I, 'X': cirq.H}
encode_gates = {0: cirq.I, 1: cirq.X}


In [ ]:
#Number of qubit for key's strength
num_bits = 33 #taking 33 as the first bit will be removed in process and we require key of length 32 for Fernet

In [ ]:
# Randon key for encrption, for the sender or simply Alice's Key
alice_key = choices([0, 1], k = num_bits)

print(alice_key)

[0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0]


In [ ]:
# JSON data to be encrypted as a string
json_data = '{"name": "Krishna", "age": 23}'

In [ ]:
# Alice's part of BB84
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')
alice_bases = choices(['Z', 'X'], k = num_bits)
alice_circuit = cirq.Circuit()
for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print(alice_circuit)

q0: ────I───H───

q1: ────I───H───

q2: ────X───I───

q3: ────X───H───

q4: ────I───I───

q5: ────I───H───

q6: ────X───I───

q7: ────X───I───

q8: ────X───I───

q9: ────X───I───

q10: ───I───I───

q11: ───I───I───

q12: ───X───I───

q13: ───X───I───

q14: ───I───H───

q15: ───I───H───

q16: ───I───I───

q17: ───I───I───

q18: ───I───I───

q19: ───I───I───

q20: ───I───H───

q21: ───X───I───

q22: ───I───H───

q23: ───X───H───

q24: ───X───I───

q25: ───I───I───

q26: ───X───H───

q27: ───X───H───

q28: ───X───I───

q29: ───X───H───

q30: ───I───I───

q31: ───I───I───

q32: ───I───I───


In [ ]:
# Bob's part of BB84
bob_bases = choices(['Z', 'X'], k=num_bits)

# Step 6: Bob measures qubits
bob_circuit = cirq.Circuit()
for bit in range(num_bits):
    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))

bob_circuit.append(cirq.measure(qubits, key = 'bob key'))
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]


In [ ]:
# Getting final key of Alice and Bob for encryption and decryption according to the BB84 Algo

final_alice_key = []
final_bob_key = []

def get_final_key(final_alice_key,final_bob_key):
  for bit in range(num_bits):
    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

# Step : Checking for an Eavesdropper
  if final_alice_key[0] == final_bob_key[0]:
    final_alice_key = final_alice_key[1:]
    final_bob_key = final_bob_key[1:]
    print('We can use our keys!')
  else:
    print('Eve was listening, we need to use a different channel!')


get_final_key(final_alice_key,final_bob_key)


We can use our keys!


In [ ]:
# Use the Alice's key for Fernet encryption suite on Alice side
alice_key_bytes=bytes(final_alice_key) #''.join(str(x) for x in alice_key)
alice_derived_key = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    iterations=100000,
    salt=alice_key_bytes,
    length=32,
).derive(alice_key_bytes)
print(final_alice_key)
alice_key_string = base64.urlsafe_b64encode(alice_derived_key) #alice_key_string + '=' * (4 - len(alice_key_string) % 4)
print(alice_key_string)
cipher_suite_alice = Fernet(alice_key_string)


# Encrypt the JSON data
encrypted_data = cipher_suite_alice.encrypt(json_data.encode())


[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
b'NvGR_g4BOQcbudIlNJfG2wZsiDiTtuiX8wmn_AC7Q3E='


In [ ]:
# Use the Bob's key for Fernet encryption suite on Bob side
bob_key_bytes=bytes(final_bob_key) # chaning the original key to bytes so support fenets requirement
bob_derived_key = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    iterations=100000,
    salt=bob_key_bytes,
    length=32,
).derive(bob_key_bytes)  # getting a key by salting the bob_key_bytes as need fernet's key to be 32 url-safe base64-encoded bytes"
bob_key_string = base64.urlsafe_b64encode(bob_derived_key)
print(bob_key_string)
cipher_suite_bob = Fernet(bob_key_string)

# Bob decrypting the encrytped data with the help of his key.
decrypted_data = cipher_suite_bob.decrypt(encrypted_data).decode()

b'NvGR_g4BOQcbudIlNJfG2wZsiDiTtuiX8wmn_AC7Q3E='


In [ ]:
#viewing original data, encrpted data, and decrpyted data.
print("Original data:",json_data)
print("Encrypted data:", encrypted_data)
print("Decrypted data (original JSON):", decrypted_data)

Original data: {"name": "Krishna", "age": 23}
Encrypted data: b'gAAAAABmJUC44gRkS7Sf6fLnnOoYYO6Wk-FM4LMC-UTU1qMZRbZ8bthm6GkfsJLkEvVdIHGyvEgbDvRrdJQbeWNQ9-f66bUMTGECNVzImgoKZORNTMp-qAM='
Decrypted data (original JSON): {"name": "Krishna", "age": 23}


```
THANK YOU, FOR THIS AMAZING INTRODUCTORY COURSE ON QUANTUM COMPUTING AND SCHOLRSHIP TO PURSUE IT.
````

```
ENTANGLED IN EMOTION OF JOY AND SORROW
```